# Script for creating synthetic, 3D mixture-of-Gaussians data

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import h5py
from scipy import stats
from IPython.display import HTML

### Parameters

In [ ]:
# File to write to
OUT_FPATH = '/home/mn2822/Desktop/WormTracking/data/synthetic/gmm_data_3d.h5'

# Image size 
IMG_SIZE = [100, 50]

# Image size limits
IMG_XLIM = [0, 100]
IMG_YLIM = [0, 50]

# Number of samples
T = 50

# Sample rate (Hz)
SMP_RATE = 10

# Number of mixture components
K = 10

# Number of 'cycles' spanning worm (does not need to be integer)
N_CYCLES = 0.75

# Frequency of worm movement (Hz)
FREQ = 0.5

# Amplitude of worm movement (image units)
AMP = 12.5

# Scale of isotropic covariance matrix for GMM
COV_SCL = 5.0

# Flag for whether or not to add noise
ADD_NOISE = False

# Noise level (stddev of Gaussian noise)
NOISE_STD = 1e-4

### Create time series of mean positions

*TODO: Add z-means (these are set to middle of z-axis)*

In [4]:
# X-values of means are equally spaced; don't change in time
means_x = np.linspace(IMG_XLIM[0], IMG_XLIM[1], K + 2);
means_x = means_x[1:K+1];
means_x = np.tile(means_x, [T, 1]);

# Y-values of means oscillate in time
phases = np.linspace(0, N_CYCLES * 2 * np.pi, K)
phases = phases[:, np.newaxis]
offset = IMG_YLIM[0] + (IMG_YLIM[1] - IMG_YLIM[0]) / 2;
rads = (2 * np.pi * FREQ / SMP_RATE) * np.arange(0, T);
rads = rads[:, np.newaxis]
means_y = offset + AMP * np.sin(rads + phases.T);

### Use mean positions to create time series of GMM densities

In [5]:
def img_pdf(x, mu, sigma):
    """Compute GMM PDF for given means and variance value."""
    
    n_comp = mu.shape[0]
    coeff = np.ones((n_comp, 1)) / n_comp
    
    f_vals = np.zeros((x.shape[0], n_comp));
    for k in range(n_comp):
        rv = stats.multivariate_normal(mu[k, :], sigma * np.eye(2))
        f_vals[:, k] = rv.pdf(x)
        
    return f_vals @ coeff
    

In [6]:
# Covariance matrix is isotropic, with scale determined by parameter
sigma = COV_SCL * np.eye(2);


# TODO: Replace this with np.mgrid formulation and change to 3D
# Create grid for evaluating densities on
xs = np.linspace(IMG_XLIM[0], IMG_XLIM[1], IMG_SIZE[0]);
ys = np.linspace(IMG_YLIM[0], IMG_YLIM[1], IMG_SIZE[1]);
[yg, xg] = np.meshgrid(ys, xs);
grid = np.hstack((xg.reshape(-1, 1), yg.reshape(-1, 1)))


# Evaluate densities to get sequence of images
data = np.zeros((IMG_SIZE[0], IMG_SIZE[1], T));
for t in range(T):
    
    # Collect means for time t into vector
    mu_x = means_x[t, :]
    mu_y = means_y[t, :]
    mu = np.hstack((mu_x[:, np.newaxis], mu_y[:, np.newaxis]))
    
    # Compute GM PDF values at grid points
    px = img_pdf(grid, mu, 5)
    
    # Normalize PDF values so image is true distribution
    px_norm = px / sum(px)
     
    # Reshape PDF vector into 2D image
    data[:, :, t] = px_norm.reshape(IMG_SIZE)

### Play synthetic data as video

In [7]:
# Create list of image plots
fig = plt.figure()
ims = []
for t in range(T):
    im = plt.imshow(data[:, :, t].T, animated=True)
    ims.append([im])
    
# Compile images into animation object
ani = animation.ArtistAnimation(
    fig, ims, interval=150, blit=True, repeat_delay=1000)

# Prevent double-display of animation
plt.close()

# Display animation in notebook
HTML(ani.to_html5_video())

### Save data to H5 file

In [8]:
with h5py.File(OUT_FPATH, 'w') as f:
    f.create_dataset('red', data=data)
    f.attrs['source'] = 'create_gmm_data_2d.ipynb'